# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [1]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
import sqlite3
from datetime import timedelta
import operator

db_filename = 'readonly/dinofunworld.db'
conn = sqlite3. connect(db_filename) 
cursor = conn.cursor()

cursor.execute("SELECT name FROM checkin, attraction WHERE checkin.attraction = attraction.attractionid GROUP BY name ORDER BY count(*) DESC limit 1")
print(cursor.fetchone()[0])

Atmosfear


In [2]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
cursor.execute("SELECT attractionid from attraction WHERE LOWER(category) like LOWER('%ride%')")

duration_dict = {}
for i in cursor.fetchall():
    ride_id = i[0]
    cursor.execute("SELECT duration FROM checkin WHERE attraction=" + str(ride_id))
    
    ride_duration = 0    
    for j in cursor.fetchall():
        count = 0
        try:
            ride_duration += timedelta(int(j[0].split(':')[0]), int(j[0].split(':')[1]), int(j[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
    duration_dict[ride_id] = ride_duration

attraction_id = max(duration_dict.items(), key=operator.itemgetter(1))[0]
cursor.execute("SELECT Name FROM attraction WHERE AttractionID=" + str(attraction_id))
print(cursor.fetchone()[0])

Flight of the Swingodon


In [3]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
cursor.execute("SELECT name FROM attraction, checkin WHERE attraction.attractionid = checkin.attraction and attraction.type='Fast Food' GROUP BY name order by count(*) asc limit 1")
print(cursor.fetchone()[0])

Theresaur Food Stop


In [4]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
cursor.execute("SELECT attractionid from attraction WHERE LOWER(category) like LOWER('%ride%')")

ride_dict = {}
for i in cursor.fetchall():
    ride_id = i[0]
    cursor.execute("SELECT duration FROM checkin WHERE attraction=" + str(ride_id))
    
    ride_duration = 0    
    for j in cursor.fetchall():
        count = 0
        try:
            ride_duration += timedelta(int(j[0].split(':')[0]), int(j[0].split(':')[1]), int(j[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
    ride_dict[ride_id] = [ride_duration]
    
    cursor.execute("SELECT count(*) FROM checkin WHERE attraction=" + str(ride_id))
    checkin_count = cursor.fetchone()[0]
    ride_dict[ride_id].append(checkin_count)
    
visit_ride = list(ride_dict.keys())[0]
wait_ride = visit_ride
both_ride = visit_ride
for ride in ride_dict:
    if ride_dict[ride][0] < ride_dict[wait_ride][0]:
        wait_ride = ride
    if ride_dict[ride][1] > ride_dict[visit_ride][1]:
        visit_ride = ride
    if (ride_dict[ride][0] < ride_dict[both_ride][0]) and (ride_dict[ride][1] > ride_dict[both_ride][1]):
        both_ride = ride
        
cursor.execute("SELECT name FROM attraction WHERE attractionid in (" + str(visit_ride) + "," + str(wait_ride) + "," + str(both_ride) + ")")
res = cursor.fetchall()
print([item[0] for item in res])

['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
